# Modeling

---


### 1)choose model


### 2)Train the model


### 3)Evaluate the model


### 4)hyperparameter tuning


### 5) final evaluation


### 6)Interpretation of model parameters


### 7) final model


# Model deployment

---
